In [9]:
import pandas as pd

# 讀取使用者上傳的 residual 檔案
file_path = "fama_residual_series.csv"
residual_df = pd.read_csv(file_path, index_col=0, parse_dates=True)

# 顯示前幾列查看格式與資料情況
residual_df.tail()

,1101 台泥,1102 亞泥,1103 嘉泥,1104 環泥,1108 幸福,1109 信大,1110 東泥,1201 味全,1203 味王,1210 大成,...,9939 宏全,9940 信義,9941 裕融,9942 茂順,9943 好樂迪,9944 新麗,9945 潤泰新,9946 三發地產,9955 佳龍,9958 世紀鋼
年月日,,,,,,,,,,,,,,,,,,,,,
2011-12-26,-0.078154,-0.075410,-0.085005,-0.086593,-0.067144,-0.074850,-0.090750,-0.080064,-0.082396,-0.079939,...,-0.074285,-0.074303,-0.073437,-0.065376,-0.087657,-0.062412,-0.063410,-0.035741,-0.081949,-0.057621
2011-12-27,-0.067753,-0.073330,-0.073321,-0.078792,-0.086819,-0.066841,-0.067056,-0.074993,-0.069717,-0.078327,...,-0.084369,-0.086299,-0.075199,-0.062897,-0.077469,-0.067890,-0.075918,-0.085144,-0.075760,-0.064474
2011-12-28,-0.076974,-0.064271,-0.052311,-0.053872,-0.069534,-0.046549,-0.042606,-0.077305,-0.054987,-0.070293,...,-0.068686,-0.059909,-0.067276,-0.046882,-0.062163,-0.060672,-0.076618,-0.083043,-0.067594,-0.067136
2011-12-29,-0.070670,-0.061436,-0.073682,-0.070828,-0.068377,-0.073454,-0.081362,-0.056655,-0.072570,-0.067535,...,-0.055253,-0.063793,-0.061268,-0.056991,-0.061453,-0.044907,-0.057955,-0.040268,-0.060066,-0.063833
2011-12-30,-0.036521,-0.031159,-0.038733,-0.037073,-0.049446,-0.025370,-0.030869,-0.038458,-0.030734,-0.029234,...,-0.056016,-0.047493,-0.047734,-0.038650,-0.043789,-0.045825,-0.046791,-0.062030,-0.062159,-0.033159


In [2]:
import numpy as np

# 將 DataFrame 轉為 numpy array，先簡化為「每檔股票為獨立樣本」
lookback = 30

X_list, y_list = [], []
dates = residual_df.index

for stock in residual_df.columns:
    series = residual_df[stock].dropna().values
    if len(series) <= lookback:
        continue
    for i in range(lookback, len(series) - 1):
        X_window = series[i - lookback:i]
        y_target = series[i]  # t+1 的 residual (i 是下一天)
        X_list.append(X_window)
        y_list.append(y_target)

X = np.array(X_list).reshape(-1, lookback, 1)
y = np.array(y_list).reshape(-1)



X.shape, y.shape

((537917, 30, 1), (537917,))

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# === 1. Sharpe Ratio 損失函數 ===
def sharpe_loss(y_true, y_pred):
    pnl = y_pred * y_true
    mean = tf.reduce_mean(pnl)
    std = tf.math.reduce_std(pnl)
    sharpe = mean / (std + 1e-6)
    return -sharpe

# === 2. 模型架構 ===
def build_model(input_len=30, cnn_filters=8, attn_heads=2, ff_dim=16):
    inp = layers.Input(shape=(input_len, 1))

    # CNN 層
    x = layers.Conv1D(filters=cnn_filters, kernel_size=3, padding='same', activation='relu')(inp)

    # Multi-Head Self Attention
    attn_out = layers.MultiHeadAttention(num_heads=attn_heads, key_dim=cnn_filters)(x, x)
    x = layers.Add()([x, attn_out])  # 殘差連接
    x = layers.LayerNormalization()(x)

    # FFN 映射成交易權重
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(ff_dim, activation='relu')(x)
    out = layers.Dense(1)(x)

    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss=sharpe_loss)
    return model

# === 3. 建立與訓練模型 ===
model = build_model()
model.fit(X, y, epochs=3, batch_size=32)

Epoch 1/3
16810/16810 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: -1.4584
Epoch 2/3
16810/16810 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: -1.8422
Epoch 3/3
16810/16810 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: -1.8800


In [4]:
pred_signal = model.predict(X)  # shape = (N, 1)

16810/16810 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step


In [5]:
strategy_return = pred_signal.flatten() * y  # y 是 t+1 residual，表示真實報酬

In [8]:
import numpy as np

pnl = strategy_return
mean_daily = np.mean(pnl)
std_daily = np.std(pnl)
sharpe =( mean_daily / (std_daily) )* np.sqrt(252)

print("年化 Sharpe:", sharpe)
print("年化報酬率:", mean_daily * 252)
print("最大單日虧損:", np.min(pnl))

年化 Sharpe: 27.104150115841218
年化報酬率: 17.838451832572243
最大單日虧損: -1.1959224867429596
